# Exp6: 基于集成学习的 Amazon 用户评论质量预测

## 一、案例简介

随着电商平台的兴起，以及疫情的持续影响，线上购物在我们的日常生活中扮演着越来越重要的角色。在进行线上商品挑选时，评论往往是我们十分关注的一个方面。然而目前电商网站的评论质量参差不齐，甚至有水军刷好评或者恶意差评的情况出现，严重影响了顾客的购物体验。因此，对于评论质量的预测成为电商平台越来越关注的话题，如果能自动对评论质量进行评估，就能根据预测结果避免展现低质量的评论。本案例中我们将基于集成学习的方法对 Amazon 现实场景中的评论质量进行预测。

## 二、作业说明

本案例中需要大家完成两种集成学习算法的实现（Bagging、AdaBoost.M1），其中基分类器要求使用 SVM 和决策树两种，因此，一共需要对比四组结果（[AUC](https://scikit-learn.org/stable/modules/model_evaluation.html#roc-metrics) 作为评价指标）：

* Bagging + SVM
* Bagging + 决策树
* AdaBoost.M1 + SVM
* AdaBoost.M1 + 决策树

注意集成学习的核心算法需要**手动进行实现**，基分类器可以调库。

### 基本要求
* 根据数据格式设计特征的表示
* 汇报不同组合下得到的 AUC (要求 -- 4种Ensemble model最佳结果AUC>0.7，最差结果AUC>0.55)
* 结合不同集成学习算法的特点分析结果之间的差异
* （使用 sklearn 等第三方库的集成学习算法会酌情扣分）

### 扩展要求
* 尝试其他基分类器（如 k-NN、朴素贝叶斯）
* 分析不同特征的影响
* 分析集成学习算法参数的影响

## 三、数据概览

In [11]:
import pandas as pd 
train_df = pd.read_csv('./data/train.csv', sep='\t')

In [12]:
test_df = pd.read_csv('./data/test.csv', sep='\t')

In [13]:
ground_truth = pd.read_csv("./data/groundTruth.csv",sep=',') # 测试集的标签

In [14]:
train_df

,reviewerID,asin,reviewText,overall,votes_up,votes_all,label
0,7885,3901,"First off, allow me to correct a common mistak...",5.0,6,7,0
1,52087,47978,I am really troubled by this Story and Enterta...,3.0,99,134,0
2,5701,3667,A near-perfect film version of a downright glo...,4.0,14,14,1
3,47191,40892,Keep your expectations low. Really really low...,1.0,4,7,0
4,40957,15367,"""they dont make em like this no more...""well.....",5.0,3,6,0
...,...,...,...,...,...,...,...
57034,58315,29374,"If you like beautifully shot, well acted films...",2.0,12,21,0
57035,23328,45548,This is a great set of films Wayne did Fox and...,5.0,15,18,0
57036,27203,42453,It's what's known as a comedy of manners. It's...,3.0,4,5,0
57037,33992,44891,Ellen can do no wrong as far a creating wonder...,5.0,4,5,0


In [15]:
test_df

,Id,reviewerID,asin,reviewText,overall
0,0,82947,37386,I REALLY wanted this series but I am in SHOCK ...,1.0
1,1,10154,23543,I have to say that this is a work of art for m...,4.0
2,2,5789,5724,Alien 3 is certainly the most controversal fil...,3.0
3,3,9198,5909,"I love this film...preachy? Well, of course i...",5.0
4,4,33252,21214,Even though I previously bought the Gamera Dou...,5.0
...,...,...,...,...,...
11203,11203,18250,35309,I honestly never heard of the graphic novel un...,5.0
11204,11204,3200,2130,Archie Bunker's command to stifle YOURSELF! wa...,5.0
11205,11205,37366,41971,"In LSD - My Problem Child, Albert Hoffman wrot...",5.0
11206,11206,1781,33089,I have owned this DVD for over a year now and ...,5.0


In [16]:
ground_truth

,Id,Expected
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
11203,11203,0
11204,11204,0
11205,11205,0
11206,11206,0


In [17]:
ground_truth.Expected.value_counts()

Expected
0    9064
1    2144
Name: count, dtype: int64

In [18]:
train_df.label.value_counts()

label
0    44137
1    12902
Name: count, dtype: int64

本次数据来源于 Amazon 电商平台，包含超过 50,000 条用户在购买商品后留下的评论，各列的含义如下：

* reviewerID：用户 ID
* asin：商品 ID
* reviewText：英文评论文本
* overall：用户对商品的打分（1-5）
* votes_up：认为评论有用的点赞数（只在训练集出现）
* votes_all：该评论得到的总评价数（只在训练集出现）
* label：评论质量的 label，1 表示高质量，0 表示低质量（只在训练集出现）

评论质量的 label 来自于其他用户对评论的 votes，votes_up/votes_all ≥ 0.9 的作为高质量评论。此外测试集包含一个额外的列 ID，标识了每一个测试的样例。

**一些tips**：
- 处理文本特征：sklearn.feature_extraction.text.TfidfVectorizer
- 大矩阵的处理：scipy.sparse
- SVM的运算速度较慢：用linearSVC代替SVC
- Ensemble的基类方法最好能输出probability而不是二分类结果，便于提升集成效果：CalibratedClassifierCV

## 四、实验

### 第一版代码：

（代码见ensemble_before_improvements.py）

这一版运行速度极慢，计算结果如下：

验证集这里 AUC ≈ 1.0 明显过拟合，而且测试集的AUC结果差距很大，具体分析如下：
- 决策树可以记住特定用词，特定长度，特定表达模式等，如果同一个reviewer的相似评价同时出现在训练集和验证集上，却不出现在测试集，就会有这个结果，线性SVM反而结果更好

### 最后一版修改

#### 第一版到最后一版的修改，主要针对以下几个点：
- 为避免出现第一版的问题，确保同一reviewerID或者ASIN（亚马逊商品标准ID）不同时出现在训练集和验证机，使用分组切分（GroupShuffleSplit）做验证：

（代码见ensemble.py）

- 在构造结构化特征时，votes_up，votes_all，helpful_ratio 这些特征和标签强相关，导致模型在验证阶段仍然产生信息泄漏。

改为：

- 在以上两点都修改之后，运算速度（尤其是包含决策树的模型）还是极慢，借鉴工程上常见的做法，即：
    - SVM 用 TF-IDF
    - 树用结构化特征（或降维后的文本特征）

改为：

运算结果如下：

## 五、实验结果与分析

| 模型                                                                         | Validation AUC | Test AUC |   |   |   |   |   |   |   |
|----------------------------------------------------------------------------|----------------|----------|---|---|---|---|---|---|---|
| AdaBoost + Decision Tree                                                   | 0.7445         | 0.7499   |   |   |   |   |   |   |   |
| AdaBoost + SVM                                                             | 0.7310         | 0.7190   |   |   |   |   |   |   |   |
| Bagging + SVM                                                              | 0.6962         | 0.7017   |   |   |   |   |   |   |   |
| Bagging + Decision Tree                                                    | 0.5746         | 0.5703   |   |   |   |   |   |   |   |

验证集与测试集表现趋势一致，结果可信。

- 比较Adaboost和Bagging

Adaboost系列模型整体比Bagging表现更好。
由于评论质量受多种因素影响（文本内容、表达方式、长度、情绪强度等），决策边界具有一定复杂性。AdaBoost对比Bagging对复杂边界的适应能力更强。

| 角度         | Bagging | AdaBoost         |   |   |   |   |   |   |   |
|------------|---------|------------------|---|---|---|---|---|---|---|
| 主要目标       | 降低方差    | 降低偏差 + 关注难样本     |   |   |   |   |   |   |   |
| 训练方式       | 各模型独立训练 | 每一轮根据前一轮错误调整样本权重 |   |   |   |   |   |   |   |

- 比较Decision Tree和线性SVM

在去除 votes 特征后（标签信息泄露，需要去除），文本特征对质量的判别能力有限，而结构化特征具有非线性关系（例如长度过短或过长都可能对应低质量），树模型更擅长这类判断，而SVM为线性模型，，因此表现略低于树模型。
Bagging + Decision Tree 表现最差，是因为 Bagging 主要降低方差，而在仅使用低维结构特征时，树模型方差并不高，偏差反而成为主要限制因素。